In [2]:
import sempy.fabric as fabric
import base64
import json
import time
import os


StatementMeta(, fde2fd78-dfa7-44ef-8d0a-e3ab63608ab9, 4, Finished, Available, Finished)

## Function to get the Report definition payload

In [3]:
def get_report_definition(workspaceId:str, itemId:str ):
    #Instantiate the client
    client = fabric.FabricRestClient()
    #request the report definition
    resp = client.post(f"v1/workspaces/{workspaceId}/items/{itemId}/getDefinition")

    ## TODO - need to be able to handle throttling for production use

    if resp.status_code == 200 :
        payload = json.loads(resp.content.decode('utf-8'))
    if resp.status_code == 202 :
        operation_resp = client.get(resp.headers["Location"])
        pcnt = json.loads(operation_resp.content.decode('utf-8'))["percentComplete"]
        percentComplete = 0 if pcnt is None else int(pcnt)

        while percentComplete < 100:
            time.sleep(1)
            operation_resp = client.get(resp.headers["Location"])
            pcnt = json.loads(operation_resp.content.decode('utf-8'))["percentComplete"]
            percentComplete = 0 if pcnt is None else int(pcnt)
        final_resp = client.get(operation_resp.headers["Location"])
        payload = json.loads(final_resp.content.decode('utf-8'))

    return payload

StatementMeta(, fde2fd78-dfa7-44ef-8d0a-e3ab63608ab9, 5, Finished, Available, Finished)

## Copy a report within the same workspace

In [ ]:
wsId = fabric.get_notebook_workspace_id()
reportId = "0004e812-86df-4477-8bb0-f88d7ba17fba"

payload = get_report_definition(wsId, reportId)

In [ ]:
client = fabric.FabricRestClient()

#add a displayName and type
payload["displayName"] = "Web Edited Report - restored"
payload["type"] = "Report"

upd_resp = client.post(f"v1/workspaces/{wsId}/items", json= payload)

print(f"Report: {reportId}  Update Response: {upd_resp.status_code}") 

# Export all reports in a Workspace
## Variables

In [4]:
# source workspace id
workspaceId = fabric.get_notebook_workspace_id()
# target workspace id
target_workspace_id = "b7324374-ebef-426c-9900-142e405a5aee"
# Folder in the lakehouse to store the files
baseDir = f"/lakehouse/default/Files/definitions/{workspaceId}" 

reports = fabric.list_items('Report', workspaceId)


StatementMeta(, fde2fd78-dfa7-44ef-8d0a-e3ab63608ab9, 6, Finished, Available, Finished)

## write each report definition out to a json file in the lakehouse

In [8]:
for index, report in reports.iterrows():
    try:
        payload = get_report_definition(workspaceId,report['Id'] )
        payload["displayName"] = report["Display Name"]
        payload["type"] = "Report"
        # Convert JSON object to string
        json_str = json.dumps(payload)

        # Convert string to byte array using bytearray
        byte_array = bytearray(json_str, 'utf-8')

        filePath = os.path.join(baseDir, f"{report['Display Name']}.json")

        print(f"Saving to lake: '{filePath}'")

        os.makedirs(os.path.dirname(filePath), exist_ok=True)

        with open(filePath, "wb") as file:
            file.write(byte_array)
    except Exception as ex:
        print(f"Error downloading item '{report.Id}'. Error: '{str(ex)}'")


StatementMeta(, fde2fd78-dfa7-44ef-8d0a-e3ab63608ab9, 10, Finished, Available, Finished)

Saving to lake: '/lakehouse/default/Files/definitions/8ef345ba-ea49-4980-a4e7-99bd2f9f392d/AzSQL AdvWrksLt.json'
Saving to lake: '/lakehouse/default/Files/definitions/8ef345ba-ea49-4980-a4e7-99bd2f9f392d/FabricMonitor-DQ.json'
Saving to lake: '/lakehouse/default/Files/definitions/8ef345ba-ea49-4980-a4e7-99bd2f9f392d/FabricMonitor-Import.json'
Saving to lake: '/lakehouse/default/Files/definitions/8ef345ba-ea49-4980-a4e7-99bd2f9f392d/Adv Wrks Test.json'
Saving to lake: '/lakehouse/default/Files/definitions/8ef345ba-ea49-4980-a4e7-99bd2f9f392d/FabricMonitorCustom Workspaces.json'
Saving to lake: '/lakehouse/default/Files/definitions/8ef345ba-ea49-4980-a4e7-99bd2f9f392d/FabricMonitorCustom2 Workspaces.json'
Saving to lake: '/lakehouse/default/Files/definitions/8ef345ba-ea49-4980-a4e7-99bd2f9f392d/Adv Works DQ to SSAS.json'
Saving to lake: '/lakehouse/default/Files/definitions/8ef345ba-ea49-4980-a4e7-99bd2f9f392d/AdvWrksLt Hybrid.json'
Saving to lake: '/lakehouse/default/Files/definitions/8

## Get a list of the report definition files

In [18]:
from os import walk

files = []
for (dirpath, dirnames, filenames) in walk(baseDir):
    files.extend(filenames)
    break

StatementMeta(, fde2fd78-dfa7-44ef-8d0a-e3ab63608ab9, 20, Finished, Available, Finished)

## Write them to the target workspace

In [24]:
client = fabric.FabricRestClient()


for f in files:
    filePath = os.path.join(baseDir, f)
    with open(filePath, 'rb') as file:
        byte_array = file.read()
    definition_str = byte_array.decode('utf-8')
    definition_json = json.loads(definition_str)

    upd_resp = client.post(f"v1/workspaces/{target_workspace_id}/items", json=definition_json, lro_wait=True)

    print(f"Report: {f} Update Response: {upd_resp.status_code}") 

StatementMeta(, fde2fd78-dfa7-44ef-8d0a-e3ab63608ab9, 26, Finished, Available, Finished)

Report: Adv Works DQ to SSAS.json Update Response: 202
Report: Adv Wrks Hybrid Live Connect.json Update Response: 202
Report: Adv Wrks Test.json Update Response: 202
Report: AdvWrksLt Hybrid.json Update Response: 202
Report: AzSQL AdvWrksLt.json Update Response: 202
Report: FabricMonitor-DQ.json Update Response: 202
Report: FabricMonitor-Import.json Update Response: 202
Report: FabricMonitorCustom Workspaces.json Update Response: 202
Report: FabricMonitorCustom2 Workspaces.json Update Response: 202
Report: Web Edited Report - restored.json Update Response: 202
Report: Web Edited Report.json Update Response: 202
